In [10]:
pip install torch-sparse


     ---------------------------------------- 0.0/210.0 kB ? eta -:--:--
     ------------- ------------------------- 71.7/210.0 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 210.0/210.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp311-cp311-win_amd64.whl size=714168 sha256=7e1bc98711fded9ef8a0788259a63cab6a5225e0fe4213a36e1acdae8090a7ba
  Stored in directory: c:\users\05amr\appdata\local\pip\cache\wheels\75\e2\1e\299c596063839303657c211f587f05591891cc6cf126d94d21
Successfully built torch-sparse


In [11]:
pip install torch-geometric[pyg-lib]

In [18]:
pip install torch-sparse

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --upgrade torch torchvision torchaudio torch-geometric

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/13/24/23cdf7e7dc33e5c01588c315f8424d31afa9edb05a80168f3d44f7178ff7/torchvision-0.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/3e/c4/bfaa620bba2e53e7707295bcc07b65871a52efe84510340ad79ce6a69be0/torchaudio-2.1.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.1 MB 3.3 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.1 MB 3.5 MB/s eta 0:00:01
   --------------------- ------------------ 0.6/1.1 MB 4.9 MB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.1 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/2.3

In [3]:
pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch-scatter

     ---------------------------------------- 0.0/108.0 kB ? eta -:--:--
     ---------------------- ---------------- 61.4/108.0 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 108.0/108.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp311-cp311-win_amd64.whl size=332371 sha256=7b01327b61df5d6fdc5b453e7629a899537c906ecae876815974917d78df6b3d
  Stored in directory: c:\users\05amr\appdata\local\pip\cache\wheels\b8\d4\0e\a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
Successfully built torch-scatter
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch_sparse

In [ ]:
import torch
from model import GAT
from dataset import AMLtoGraph
import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = AMLtoGraph("C:\\Users\\05amr\\Downloads\\AntiMoneyLaunderingDetectionWithGNN\\AntiMoneyLaunderingDetectionWithGNN-main\\data")
data = dataset[0]
epoch = 100

model = GAT(in_channels=data.num_features, hidden_channels=16, out_channels=1, heads=8)
model = model.to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

split = T.RandomNodeSplit(split='train_rest', num_val=0.1, num_test=0)
data = split(data)

train_loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data.train_mask,
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data.val_mask,
)

for i in range(epoch):
    total_loss = 0
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        data.to(device)
        pred = model(data.x, data.edge_index, data.edge_attr)
        ground_truth = data.y
        loss = criterion(pred, ground_truth.unsqueeze(1))
        loss.backward()
        optimizer.step()
        total_loss += float(loss)
    if epoch%10 == 0:
        print(f"Epoch: {i:03d}, Loss: {total_loss:.4f}")
        model.eval()
        acc = 0
        total = 0
        with torch.no_grad():
            for test_data in test_loader:
                test_data.to(device)
                pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
                ground_truth = test_data.y
                correct = (pred == ground_truth.unsqueeze(1)).sum().item()
                total += len(ground_truth)
                acc += correct
            acc = acc/total
            print('accuracy:', acc)

Epoch: 000, Loss: 11662.7070
accuracy: 0.9555176257489202
Epoch: 001, Loss: 6454.0015
accuracy: 0.9700079162372481
Epoch: 002, Loss: 4663.3213
accuracy: 0.971046150018659
Epoch: 003, Loss: 3441.4109
accuracy: 0.97180280877069
Epoch: 004, Loss: 2890.0322
accuracy: 0.9721659389950664
Epoch: 005, Loss: 2623.5172
accuracy: 0.9722039702924955
Epoch: 006, Loss: 2460.1582
accuracy: 0.9725538596290416
Epoch: 007, Loss: 2366.4748
accuracy: 0.9727851535470896
Epoch: 008, Loss: 2270.8517
accuracy: 0.9726900273298852
Epoch: 009, Loss: 2225.7713
accuracy: 0.972798901005918
Epoch: 010, Loss: 2171.5822
accuracy: 0.9730330817416052
Epoch: 011, Loss: 2150.8398
accuracy: 0.9730969822438092
Epoch: 012, Loss: 2085.7716
accuracy: 0.9730979193932849
Epoch: 013, Loss: 2041.1031
accuracy: 0.9732607808190535
Epoch: 014, Loss: 1993.9662
accuracy: 0.9733210488308972
Epoch: 015, Loss: 1975.8263
accuracy: 0.9733633277923216
